[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danpele/Time-Series-Analysis/blob/main/EN/Course_Notebooks/chapter9_lecture_notebook.ipynb)

---

# Chapter 9: Prophet & TBATS for Multiple Seasonality

**Course:** Time Series Analysis and Forecasting  
**Program:** Bachelor program, Faculty of Cybernetics, Statistics and Economic Informatics, Bucharest University of Economic Studies, Romania  
**Academic Year:** 2025-2026

---

## Learning Objectives

By the end of this notebook, you will be able to:
1. Understand why standard SARIMA fails for multiple seasonality
2. Apply TBATS for complex seasonal patterns
3. Use Facebook Prophet for forecasting with holidays
4. Compare performance of Prophet vs TBATS
5. Handle multiple seasonal periods (daily, weekly, annual)
6. Interpret model components and diagnostics

## Setup and Imports

In [ ]:
# Install required packages (uncomment if needed in Colab)
# !pip install prophet tbats statsmodels pandas numpy matplotlib -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Statistical models
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Prophet
try:
    from prophet import Prophet
    HAS_PROPHET = True
except ImportError:
    HAS_PROPHET = False
    print("Prophet not installed. Install with: pip install prophet")

# TBATS
try:
    from tbats import TBATS
    HAS_TBATS = True
except ImportError:
    HAS_TBATS = False
    print("TBATS not installed. Install with: pip install tbats")

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Plotting style
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666'}

print("Setup complete!")
print(f"Prophet available: {HAS_PROPHET}")
print(f"TBATS available: {HAS_TBATS}")

## 1. The Multiple Seasonality Challenge

### Why SARIMA Fails

Standard SARIMA can only handle **one** seasonal period (parameter $m$).

Many real-world time series have **multiple** overlapping seasonal patterns:
- **Hourly electricity demand**: daily (24h) + weekly (168h) + annual (8760h)
- **Retail sales**: weekly + monthly + annual
- **Web traffic**: daily + weekly + holiday effects

In [ ]:
# REAL DATA: Air Passengers (1949-1960) - Classic time series with trend + seasonality
# This is one of the most famous time series datasets in statistics

# Air Passengers monthly data (thousands of passengers)
air_passengers_data = [
    112, 118, 132, 129, 121, 135, 148, 148, 136, 119, 104, 118,  # 1949
    115, 126, 141, 135, 125, 149, 170, 170, 158, 133, 114, 140,  # 1950
    145, 150, 178, 163, 172, 178, 199, 199, 184, 162, 146, 166,  # 1951
    171, 180, 193, 181, 183, 218, 230, 242, 209, 191, 172, 194,  # 1952
    196, 196, 236, 235, 229, 243, 264, 272, 237, 211, 180, 201,  # 1953
    204, 188, 235, 227, 234, 264, 302, 293, 259, 229, 203, 229,  # 1954
    242, 233, 267, 269, 270, 315, 364, 347, 312, 274, 237, 278,  # 1955
    284, 277, 317, 313, 318, 374, 413, 405, 355, 306, 271, 306,  # 1956
    315, 301, 356, 348, 355, 422, 465, 467, 404, 347, 305, 336,  # 1957
    340, 318, 362, 348, 363, 435, 491, 505, 404, 359, 310, 337,  # 1958
    360, 342, 406, 396, 420, 472, 548, 559, 463, 407, 362, 405,  # 1959
    417, 391, 419, 461, 472, 535, 622, 606, 508, 461, 390, 432   # 1960
]

dates = pd.date_range(start='1949-01-01', periods=len(air_passengers_data), freq='MS')
df = pd.DataFrame({'ds': dates, 'y': air_passengers_data})
df.set_index('ds', inplace=True)

print("REAL DATA: Air Passengers (1949-1960)")
print("="*50)
print(f"Period: {df.index[0].strftime('%Y-%m')} to {df.index[-1].strftime('%Y-%m')}")
print(f"Observations: {len(df)} months")
print(f"\nData characteristics:")
print(f"  - Clear upward TREND (growth in air travel)")
print(f"  - YEARLY seasonality (s=12): Summer peaks")
print(f"  - MULTIPLICATIVE: Seasonal amplitude grows with level")
print(f"\nNote: This classic dataset demonstrates why we need")
print(f"      models that handle both trend AND seasonality!")

In [ ]:
# Visualize the Air Passengers data
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Full series
axes[0].plot(df.index, df['y'], color=COLORS['blue'], linewidth=1.5)
axes[0].set_title('Air Passengers (1949-1960): Full Series', fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Passengers (thousands)')

# Seasonal pattern - compare same months across years
years = df.index.year.unique()
for year in years[::2]:  # Every other year
    year_data = df[df.index.year == year]
    axes[1].plot(year_data.index.month, year_data['y'].values, 
                 label=str(year), linewidth=1.5, alpha=0.7)
axes[1].set_title('Yearly Seasonality: Summer Peaks (July-August)', fontweight='bold')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Passengers (thousands)')
axes[1].set_xticks(range(1, 13))
axes[1].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=6, frameon=False)

# Monthly averages showing seasonal pattern
monthly_avg = df.groupby(df.index.month)['y'].mean()
axes[2].bar(monthly_avg.index, monthly_avg.values, color=COLORS['orange'], alpha=0.7)
axes[2].set_title('Average Monthly Pattern: Peak Travel Season', fontweight='bold')
axes[2].set_xlabel('Month')
axes[2].set_ylabel('Average Passengers')
axes[2].set_xticks(range(1, 13))
axes[2].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

plt.tight_layout()
plt.subplots_adjust(bottom=0.1)
plt.show()

print("Key insight: Notice how seasonal amplitude GROWS over time!")
print("Early years: peaks ~30 passengers difference")
print("Later years: peaks ~200 passengers difference")
print("This is MULTIPLICATIVE seasonality!")

## 2. TBATS: Trigonometric Seasonality

### What TBATS Stands For

- **T**rigonometric: Fourier terms for seasonality
- **B**ox-Cox: Variance stabilization
- **A**RMA: Error autocorrelation
- **T**rend: Damped local linear trend
- **S**easonal: Multiple seasonal periods

### Fourier Representation of Seasonality

$$s_t^{(i)} = \sum_{j=1}^{k_i} \left[ a_j^{(i)} \cos\left(\frac{2\pi j t}{m_i}\right) + b_j^{(i)} \sin\left(\frac{2\pi j t}{m_i}\right) \right]$$

Where:
- $m_i$ = seasonal period $i$
- $k_i$ = number of harmonics (complexity of shape)
- Maximum $k_i = m_i / 2$

In [ ]:
# Demonstrate Fourier terms for seasonality
t = np.linspace(0, 2*np.pi, 100)

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# K=1: Simple sine wave
k1 = np.sin(t)
axes[0, 0].plot(t, k1, color=COLORS['blue'], linewidth=2)
axes[0, 0].set_title('K=1: Simple Sinusoid', fontweight='bold')
axes[0, 0].set_xlabel('Time')

# K=2: Can capture asymmetry
k2 = np.sin(t) + 0.5*np.sin(2*t)
axes[0, 1].plot(t, k2, color=COLORS['orange'], linewidth=2)
axes[0, 1].set_title('K=2: More Flexible (asymmetric peaks)', fontweight='bold')
axes[0, 1].set_xlabel('Time')

# K=5: Complex patterns
k5 = np.sin(t) + 0.5*np.sin(2*t) + 0.3*np.sin(3*t) + 0.2*np.sin(4*t) + 0.1*np.sin(5*t)
axes[1, 0].plot(t, k5, color=COLORS['green'], linewidth=2)
axes[1, 0].set_title('K=5: Complex Pattern (sharp features)', fontweight='bold')
axes[1, 0].set_xlabel('Time')

# Comparison
axes[1, 1].plot(t, k1, label='K=1', linewidth=2)
axes[1, 1].plot(t, k2, label='K=2', linewidth=2)
axes[1, 1].plot(t, k5, label='K=5', linewidth=2)
axes[1, 1].set_title('Comparison: More K = More Flexibility', fontweight='bold')
axes[1, 1].set_xlabel('Time')
axes[1, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)

plt.tight_layout()
plt.subplots_adjust(bottom=0.12)
plt.show()

print("Key: K harmonics = K pairs of (sin, cos) terms")
print("More K = more complex seasonal shapes (but risk of overfitting)")

In [ ]:
# Prepare data for modeling (shared by TBATS and Prophet)
df_monthly = df.copy()

# Train/test split (use last 24 months for testing)
train_size = len(df_monthly) - 24
train = df_monthly.iloc[:train_size]
test = df_monthly.iloc[train_size:]

print("Data Preparation for TBATS and Prophet")
print("="*50)
print(f"Training period: {train.index[0].strftime('%Y-%m')} to {train.index[-1].strftime('%Y-%m')}")
print(f"Test period: {test.index[0].strftime('%Y-%m')} to {test.index[-1].strftime('%Y-%m')}")
print(f"Training samples: {len(train)}, Test samples: {len(test)}")

if HAS_TBATS:
    print(f"\nFitting TBATS with seasonal period: [12] (yearly)...")
    
    # Fit TBATS with yearly seasonality
    estimator = TBATS(seasonal_periods=[12])
    model_tbats = estimator.fit(train['y'].values)
    
    print("\nModel Summary:")
    print(model_tbats.summary())
else:
    print("\nTBATS not available. Please install with: pip install tbats")

In [ ]:
if HAS_TBATS:
    # Forecast
    forecast_horizon = len(test)
    forecast_tbats = model_tbats.forecast(steps=forecast_horizon)
    
    # Metrics
    rmse_tbats = np.sqrt(mean_squared_error(test['y'], forecast_tbats))
    mae_tbats = mean_absolute_error(test['y'], forecast_tbats)
    mape_tbats = np.mean(np.abs((test['y'].values - forecast_tbats) / test['y'].values)) * 100
    
    print("TBATS Forecast Results on Air Passengers")
    print("="*50)
    print(f"RMSE: {rmse_tbats:.2f}")
    print(f"MAE:  {mae_tbats:.2f}")
    print(f"MAPE: {mape_tbats:.1f}%")
    
    # Plot
    fig, ax = plt.subplots(figsize=(14, 5))
    
    ax.plot(train.index, train['y'], color=COLORS['blue'], label='Train', linewidth=1)
    ax.plot(test.index, test['y'], color=COLORS['green'], label='Actual', linewidth=1.5)
    ax.plot(test.index, forecast_tbats, color=COLORS['red'], label='TBATS Forecast', linewidth=1.5, linestyle='--')
    ax.axvline(x=test.index[0], color='black', linestyle=':', alpha=0.5)
    ax.set_title('TBATS Forecast vs Actual (Air Passengers)', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Passengers (thousands)')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=3, frameon=False)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.18)
    plt.show()

## 3. Prophet: Facebook's Forecasting Tool

### Prophet Decomposition

Prophet uses an additive decomposition:

$$y(t) = g(t) + s(t) + h(t) + \varepsilon_t$$

Where:
- $g(t)$ = **Trend** (linear or logistic growth with changepoints)
- $s(t)$ = **Seasonality** (Fourier series)
- $h(t)$ = **Holidays** (user-specified events)
- $\varepsilon_t$ = Error term

### Key Features

1. **Automatic changepoint detection** in trend
2. **Multiple seasonalities** (daily, weekly, annual)
3. **Holiday effects** (easily incorporated)
4. **Robust to missing data**
5. **Intuitive parameters** for analysts

In [ ]:
if HAS_PROPHET:
    # Prepare data for Prophet (requires 'ds' and 'y' columns)
    df_prophet = df_monthly.reset_index()
    df_prophet.columns = ['ds', 'y']
    
    # Train/test split
    train_prophet = df_prophet.iloc[:train_size]
    test_prophet = df_prophet.iloc[train_size:]
    
    print("Prophet Model Training on Air Passengers")
    print("="*50)
    
    # Initialize Prophet model
    # Note: Air Passengers has multiplicative seasonality!
    model_prophet = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,  # Monthly data, no weekly pattern
        daily_seasonality=False,   # Monthly data
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10,
        seasonality_mode='multiplicative'  # Key for Air Passengers!
    )
    
    # Fit model
    model_prophet.fit(train_prophet)
    print("Model fitted successfully!")
    print("Note: Using multiplicative seasonality mode")
else:
    print("Prophet not available. Please install with: pip install prophet")

In [ ]:
if HAS_PROPHET:
    # Create future dataframe for prediction
    future = model_prophet.make_future_dataframe(periods=len(test_prophet), freq='MS')
    
    # Forecast
    forecast_prophet = model_prophet.predict(future)
    
    # Extract test predictions
    test_predictions = forecast_prophet.iloc[train_size:]['yhat'].values
    
    # Metrics
    rmse_prophet = np.sqrt(mean_squared_error(test['y'], test_predictions))
    mae_prophet = mean_absolute_error(test['y'], test_predictions)
    mape_prophet = np.mean(np.abs((test['y'].values - test_predictions) / test['y'].values)) * 100
    
    print("Prophet Forecast Results on Air Passengers")
    print("="*50)
    print(f"RMSE: {rmse_prophet:.2f}")
    print(f"MAE:  {mae_prophet:.2f}")
    print(f"MAPE: {mape_prophet:.1f}%")

In [ ]:
if HAS_PROPHET:
    # Plot Prophet components
    fig = model_prophet.plot_components(forecast_prophet)
    plt.suptitle('Prophet Component Decomposition (Air Passengers)', fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("\nComponent Interpretation:")
    print("- Trend: Strong upward growth in air travel (1949-1960)")
    print("- Yearly: Clear summer peaks (July-August) - vacation season")
    print("- Note: Using multiplicative mode - seasonality is a multiplier!")

In [ ]:
if HAS_PROPHET:
    # Forecast plot with uncertainty
    fig, ax = plt.subplots(figsize=(14, 5))
    
    # Historical data
    ax.plot(train_prophet['ds'], train_prophet['y'], 
            color=COLORS['blue'], label='Train', linewidth=1)
    ax.plot(test_prophet['ds'], test_prophet['y'], 
            color=COLORS['green'], label='Actual', linewidth=1.5)
    
    # Forecast
    test_forecast = forecast_prophet.iloc[train_size:]
    ax.plot(test_forecast['ds'], test_forecast['yhat'], 
            color=COLORS['red'], label='Prophet Forecast', linewidth=1.5, linestyle='--')
    
    # Uncertainty interval
    ax.fill_between(test_forecast['ds'], 
                    test_forecast['yhat_lower'], 
                    test_forecast['yhat_upper'],
                    color=COLORS['red'], alpha=0.2, label='95% CI')
    
    ax.axvline(x=test_prophet['ds'].iloc[0], color='black', linestyle=':', alpha=0.5)
    ax.set_title('Prophet Forecast with Uncertainty Interval (Air Passengers)', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Passengers (thousands)')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=4, frameon=False)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.18)
    plt.show()

## 4. Adding Holiday Effects in Prophet

Prophet makes it easy to incorporate known events/holidays.

In [ ]:
if HAS_PROPHET:
    # Define holidays for the Air Passengers time period (1949-1960)
    # Note: These are US holidays that might affect air travel
    holidays_list = []
    for year in range(1949, 1961):
        holidays_list.extend([
            {'holiday': 'new_year', 'ds': f'{year}-01-01'},
            {'holiday': 'independence_day', 'ds': f'{year}-07-04'},
            {'holiday': 'thanksgiving', 'ds': f'{year}-11-25'},  # Approximate
            {'holiday': 'christmas', 'ds': f'{year}-12-25'},
        ])
    
    holidays = pd.DataFrame(holidays_list)
    holidays['ds'] = pd.to_datetime(holidays['ds'])
    holidays['lower_window'] = -1
    holidays['upper_window'] = 1
    
    print("Holiday DataFrame (sample)")
    print(holidays.head(8))
    
    # Fit model with holidays
    model_with_holidays = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        holidays=holidays,
        seasonality_mode='multiplicative'
    )
    
    model_with_holidays.fit(train_prophet)
    
    # Forecast
    future_h = model_with_holidays.make_future_dataframe(periods=len(test_prophet), freq='MS')
    forecast_h = model_with_holidays.predict(future_h)
    
    print("\nModel with holidays fitted successfully!")
    print("Holiday effect will appear in component decomposition.")

## 5. Changepoints in Prophet

Prophet automatically detects points where the trend changes slope.

In [ ]:
if HAS_PROPHET:
    # Get changepoints
    changepoints = model_prophet.changepoints
    
    print(f"Detected {len(changepoints)} changepoints:")
    print(changepoints[:5].tolist())  # Show first 5
    
    # Plot trend with changepoints
    fig, ax = plt.subplots(figsize=(14, 5))
    
    ax.plot(forecast_prophet['ds'], forecast_prophet['trend'], 
            color=COLORS['blue'], linewidth=2, label='Trend')
    
    for cp in changepoints:
        ax.axvline(x=cp, color=COLORS['red'], linestyle='--', alpha=0.3)
    
    ax.set_title('Prophet Trend with Changepoints', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Trend Value')
    ax.legend(['Trend', 'Changepoints'], loc='upper center', 
              bbox_to_anchor=(0.5, -0.12), ncol=2, frameon=False)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.18)
    plt.show()

## 6. Prophet vs TBATS Comparison

In [ ]:
if HAS_PROPHET and HAS_TBATS:
    # Compare forecasts on Air Passengers
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Forecasts comparison
    axes[0].plot(test.index, test['y'], color=COLORS['blue'], label='Actual', linewidth=1.5)
    axes[0].plot(test.index, forecast_tbats, color=COLORS['orange'], 
                 label=f'TBATS (RMSE={rmse_tbats:.1f})', linewidth=1.5, linestyle='--')
    axes[0].plot(test.index, test_predictions, color=COLORS['green'], 
                 label=f'Prophet (RMSE={rmse_prophet:.1f})', linewidth=1.5, linestyle=':')
    axes[0].set_title('Forecast Comparison (Air Passengers)', fontweight='bold')
    axes[0].set_xlabel('Date')
    axes[0].set_ylabel('Passengers (thousands)')
    axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
    
    # Metrics comparison
    metrics = ['RMSE', 'MAE', 'MAPE (%)']
    tbats_vals = [rmse_tbats, mae_tbats, mape_tbats]
    prophet_vals = [rmse_prophet, mae_prophet, mape_prophet]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    bars1 = axes[1].bar(x - width/2, tbats_vals, width, label='TBATS', color=COLORS['orange'])
    bars2 = axes[1].bar(x + width/2, prophet_vals, width, label='Prophet', color=COLORS['green'])
    
    axes[1].set_ylabel('Value')
    axes[1].set_title('Model Performance Comparison', fontweight='bold')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(metrics)
    axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
    
    # Add value labels
    for bar, val in zip(bars1, tbats_vals):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                     f'{val:.1f}', ha='center', fontsize=9)
    for bar, val in zip(bars2, prophet_vals):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                     f'{val:.1f}', ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)
    plt.show()
    
    print("\nModel Comparison Summary (Air Passengers)")
    print("="*50)
    print(f"{'Metric':<10} {'TBATS':>10} {'Prophet':>10}")
    print("-"*30)
    print(f"{'RMSE':<10} {rmse_tbats:>10.2f} {rmse_prophet:>10.2f}")
    print(f"{'MAE':<10} {mae_tbats:>10.2f} {mae_prophet:>10.2f}")
    print(f"{'MAPE (%)':<10} {mape_tbats:>10.1f} {mape_prophet:>10.1f}")

## 7. Multiplicative vs Additive Seasonality

Prophet supports both modes:
- **Additive**: Seasonal effect is constant ($y = trend + season$)
- **Multiplicative**: Seasonal effect scales with level ($y = trend \times (1 + season)$)

In [ ]:
# Demonstrate additive vs multiplicative using REAL DATA
# Air Passengers is a classic example of MULTIPLICATIVE seasonality

# For comparison, let's also use US Retail Sales data (more recent)
# US Retail Sales - Monthly data from FRED (billions of dollars, 2018-2023)
retail_sales_data = [
    457.6, 459.1, 468.2, 469.2, 473.9, 477.6, 482.1, 483.0, 473.7, 476.2, 477.9, 502.7,  # 2018
    455.6, 459.8, 472.0, 470.5, 479.3, 480.7, 485.9, 488.6, 479.9, 483.6, 481.7, 516.0,  # 2019
    461.2, 461.5, 414.7, 384.9, 476.4, 509.3, 516.1, 521.7, 527.0, 524.7, 519.6, 553.3,  # 2020
    510.6, 507.4, 560.1, 561.1, 567.0, 574.0, 582.0, 585.0, 581.0, 596.1, 595.6, 630.1,  # 2021
    581.9, 587.8, 631.5, 613.8, 629.3, 633.0, 631.8, 638.7, 625.5, 641.0, 633.7, 671.9,  # 2022
    620.6, 624.0, 670.2, 656.5, 666.3, 670.1, 673.2, 679.3, 668.6, 686.1, 672.3, 724.5   # 2023
]
retail_dates = pd.date_range(start='2018-01-01', periods=len(retail_sales_data), freq='MS')
df_retail = pd.DataFrame({'ds': retail_dates, 'y': retail_sales_data})

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Air Passengers - Multiplicative seasonality
axes[0].plot(df.index, df['y'], color=COLORS['orange'], linewidth=1)
# Add trend line
z = np.polyfit(range(len(df)), df['y'].values, 1)
p = np.poly1d(z)
axes[0].plot(df.index, p(range(len(df))), color=COLORS['red'], linewidth=2, linestyle='--', label='Trend')
axes[0].set_title('Air Passengers (1949-1960)\nMULTIPLICATIVE: Amplitude grows with level', fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Passengers (thousands)')
axes[0].legend()

# US Retail Sales - Also shows multiplicative pattern (seasonal peaks grow)
axes[1].plot(df_retail['ds'], df_retail['y'], color=COLORS['blue'], linewidth=1)
# Add trend line
z2 = np.polyfit(range(len(df_retail)), df_retail['y'].values, 1)
p2 = np.poly1d(z2)
axes[1].plot(df_retail['ds'], p2(range(len(df_retail))), color=COLORS['red'], linewidth=2, linestyle='--', label='Trend')
axes[1].set_title('US Retail Sales (2018-2023)\nMULTIPLICATIVE: December peaks grow', fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Sales ($ billions)')
axes[1].legend()

plt.tight_layout()
plt.show()

print("REAL DATA Examples of Seasonality Types:")
print("="*55)
print("\n1. Air Passengers (1949-1960):")
print("   - Early years: Summer peak ~30 above trend")
print("   - Later years: Summer peak ~200 above trend")
print("   - → MULTIPLICATIVE (amplitude scales with level)")
print("\n2. US Retail Sales (2018-2023):")
print("   - December holiday shopping peaks")
print("   - Peaks grow as overall retail grows")
print("   - COVID-19 impact visible in 2020 (March-April drop)")
print("   - → MULTIPLICATIVE seasonality")

In [ ]:
if HAS_PROPHET:
    # Fit Prophet with multiplicative seasonality on US Retail Sales
    df_retail_prophet = df_retail.copy()
    
    model_mult = Prophet(
        seasonality_mode='multiplicative',
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        changepoint_prior_scale=0.1  # More flexible for COVID impact
    )
    
    model_mult.fit(df_retail_prophet)
    future_mult = model_mult.make_future_dataframe(periods=12, freq='MS')
    forecast_mult = model_mult.predict(future_mult)
    
    # Plot components
    fig = model_mult.plot_components(forecast_mult)
    plt.suptitle('Prophet Components - US Retail Sales (Multiplicative Mode)', fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("\nUS Retail Sales Component Interpretation:")
    print("- Trend: Strong growth with COVID-19 recovery (2020-2021)")
    print("- Yearly: December peak (holiday shopping) is the dominant pattern")
    print("- Multiplicative mode captures growing seasonal amplitude")

## Summary

### Key Takeaways

1. **Multiple Seasonality** is common in real data but SARIMA can only handle one period

2. **TBATS** (Trigonometric, Box-Cox, ARMA, Trend, Seasonal):
   - Uses Fourier terms for flexible seasonal patterns
   - Automatic selection of harmonics via AIC
   - State-space formulation for efficient computation
   - Good for automatic forecasting

3. **Prophet**:
   - Decomposable model: $y = g(t) + s(t) + h(t) + \varepsilon$
   - Easy incorporation of holidays and events
   - Automatic changepoint detection
   - Interpretable components
   - Good when domain knowledge is available

### When to Use What?

| Situation | Recommendation |
|-----------|----------------|
| Known holidays/events | Prophet |
| Automatic forecasting, no domain knowledge | TBATS |
| Need interpretable components | Prophet |
| Very long seasonal periods (e.g., 8760 hours) | TBATS (more efficient) |
| Need uncertainty intervals easily | Prophet |
| Business reporting & communication | Prophet (cleaner plots) |